In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
#df= pd.read_csv('../../data/more_features/with_correlated_feature_0.1.csv')
df =pd.read_csv('../../data/more_features/more_feats_new_labels_0.1_noisy_0.4.csv')
#df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,12.190830,163.097819,3.084127,5.923027,232.339305,79.550470,-1.000000,3.748689,-1.000000,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Anemia of chronic disease
1,9.944819,187.086684,5.096256,-1.000000,-1.000000,100.121185,-1.000000,5.857746,72.837294,0,1.146823,-1.000000,-1.000000,-1.000000,-1.000000,20.943783,-1.000000,-1.000000,Unspecified anemia
2,13.703345,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,80.372015,5.684361,45.702318,0,1.789854,206.817706,1.551467,90.217875,76.009442,22.651663,12.551513,-1.000000,No anemia
3,7.346123,27.669632,-1.000000,-1.000000,428.089083,77.375356,95.912445,6.469686,52.594562,0,0.222080,297.319109,1.265341,126.999491,46.998397,77.183582,8.518821,119.113878,Anemia of chronic disease
4,12.295548,225.097199,3.827717,0.000000,-1.000000,102.137453,-1.000000,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Unspecified anemia


In [4]:
utils.get_dt_performance(df)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional in

(0.9495714285714286,
 0.9375796043606589,
 0.9649074036208941,
 datetime.timedelta(microseconds=2994))

In [5]:
df.label.value_counts()

No anemia                               10000
Anemia of chronic disease                9756
Iron deficiency anemia                   9267
Unspecified anemia                       9033
Aplastic anemia                          9020
Vitamin B12/Folate deficiency anemia     9000
Hemolytic anemia                         8976
Inconclusive diagnosis                   4948
Name: label, dtype: int64

In [6]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional in

((56000, 18), (14000, 18), (56000,), (14000,))

In [7]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'age',
 'gender',
 'indirect_bilirubin',
 'transferrin',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose']

In [ ]:
for steps in [int(8.5e6), int(9e6), int(9.5e6), int(10e6), int(10.5e6)]:
#for steps in [int(2e3)]:
    #start_time = datetime.now()
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../models/many_features/0.1/dqn3_by_type_new_labels_noisy_4_{steps}')
    #end_time = datetime.now()
    #print(f'The duration for {steps} steps is {end_time-start_time}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.64     |
|    ep_rew_mean      | -0.88    |
|    exploration_rate | 0.656    |
|    success_rate     | 0.06     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 916      |
|    time_elapsed     | 335      |
|    total_timesteps  | 307871   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 5.95     |
|    n_updates        | 64467    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.18     |
|    ep_rew_mean      | -0.98    |
|    exploration_rate | 0.237    |
|    success_rate     | 0.06     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 780      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.8      |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.05     |
|    success_rate     | 0.51     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 535      |
|    time_elapsed     | 11000    |
|    total_timesteps  | 5893494  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.194    |
|    n_updates        | 1460873  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.51     |
|    ep_rew_mean      | 0.08     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.55     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 528      |
|    time_elapsed     | 11856    |
|    total_timesteps  | 6261397  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.84     |
|    ep_rew_mean      | -0.1     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.45     |
| time/               |          |
|    episodes         | 900000   |
|    fps              | 669      |
|    time_elapsed     | 4745     |
|    total_timesteps  | 3175463  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.137    |
|    n_updates        | 781365   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.99     |
|    ep_rew_mean      | 0.16     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.6      |
| time/               |          |
|    episodes         | 1000000  |
|    fps              | 664      |
|    time_elapsed     | 5343     |
|    total_timesteps  | 3550932  |
| train/              |          |
|    learning_rate  

In [44]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/dqn3_by_type_new_labels_noisy_4_13000000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

Using stable baselines 3
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done.....


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,4.0,0.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Aplastic anemia]",6.0,6.0
1,5.0,1.0,1.0,1.0,0.0,"[hemoglobin, mcv, segmented_neutrophils, gende...",2.0,2.0
2,5.0,2.0,1.0,1.0,0.0,"[hemoglobin, mcv, segmented_neutrophils, gende...",2.0,2.0
3,4.0,3.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Hemolytic anemia]",5.0,5.0
4,4.0,4.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Hemolytic anemia]",5.0,5.0


In [45]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [46]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [47]:
test_df.y_pred.value_counts()

3.0    1979
0.0    1967
7.0    1904
4.0    1820
6.0    1665
2.0    1610
5.0    1607
1.0    1448
Name: y_pred, dtype: int64

In [48]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

87.59285714285714

In [49]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(4.144428571428572, 0.6134285714285714)

In [50]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.8759285714285714, 0.8700025801777977, 0.931552560645486)

In [51]:
test_df.y_pred.unique()

array([6., 2., 5., 0., 4., 3., 7., 1.])

In [34]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)